In [1]:
import numpy as np
import pandas as pd

In [2]:
path = 'resources/food-inspections.csv'
df = pd.read_csv(path, sep=',')
df.head(2)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2316058,CHICAGO TAICHI BUBBLE TEA,CHICAGO TAICHI BUBBLE TEA,2694548.0,Restaurant,Risk 1 (High),6800 N SHERIDAN RD,CHICAGO,IL,60626.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,42.005587,-87.661077,"{'longitude': '42.00558686485114', 'latitude':...",NaN,NaN,NaN,NaN,NaN
1,2316065,Krispy Kreme Doughnuts,Krispy Kreme Doughnuts,2689327.0,Restaurant,Risk 2 (Medium),108 N STATE ST,CHICAGO,IL,60602.0,...,Pass,NaN,41.883423,-87.628022,"{'longitude': '41.88342263701488', 'latitude':...",NaN,NaN,NaN,NaN,NaN


In [3]:
nb_NaN_violations = len(df) - df['Violations'].count()
print("We have {} out of {} inspections with no violations recorded. It makes {}%.".format(nb_NaN_violations, len(df), "%.2f" % (nb_NaN_violations/len(df)*100)))

We have 51597 out of 194615 inspections with no violations recorded. It makes 26.51%.


In [4]:
df_split = df.copy()
df_split['Violations'] = df_split['Violations'].map(lambda x: str(x).split('|'))
df_split.head(3)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2316058,CHICAGO TAICHI BUBBLE TEA,CHICAGO TAICHI BUBBLE TEA,2694548.0,Restaurant,Risk 1 (High),6800 N SHERIDAN RD,CHICAGO,IL,60626.0,...,Pass,[47. FOOD & NON-FOOD CONTACT SURFACES CLEANABL...,42.005587,-87.661077,"{'longitude': '42.00558686485114', 'latitude':...",NaN,NaN,NaN,NaN,NaN
1,2316065,Krispy Kreme Doughnuts,Krispy Kreme Doughnuts,2689327.0,Restaurant,Risk 2 (Medium),108 N STATE ST,CHICAGO,IL,60602.0,...,Pass,[nan],41.883423,-87.628022,"{'longitude': '41.88342263701488', 'latitude':...",NaN,NaN,NaN,NaN,NaN
2,2316054,SUBWAY,SUBWAY,2590201.0,Restaurant,Risk 1 (High),2512 W NORTH AVE,CHICAGO,IL,60647.0,...,Pass,[39. CONTAMINATION PREVENTED DURING FOOD PREPA...,41.910399,-87.690207,"{'longitude': '41.910398978211525', 'latitude'...",NaN,NaN,NaN,NaN,NaN


In [19]:
violations = df_split.explode('Violations').rename(columns={'Violations' : 'Violation'})
violations = violations[violations['Violation'] != 'nan']
violations.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violation,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2316058,CHICAGO TAICHI BUBBLE TEA,CHICAGO TAICHI BUBBLE TEA,2694548.0,Restaurant,Risk 1 (High),6800 N SHERIDAN RD,CHICAGO,IL,60626.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,42.005587,-87.661077,"{'longitude': '42.00558686485114', 'latitude':...",NaN,NaN,NaN,NaN,NaN
0,2316058,CHICAGO TAICHI BUBBLE TEA,CHICAGO TAICHI BUBBLE TEA,2694548.0,Restaurant,Risk 1 (High),6800 N SHERIDAN RD,CHICAGO,IL,60626.0,...,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED...",42.005587,-87.661077,"{'longitude': '42.00558686485114', 'latitude':...",NaN,NaN,NaN,NaN,NaN
2,2316054,SUBWAY,SUBWAY,2590201.0,Restaurant,Risk 1 (High),2512 W NORTH AVE,CHICAGO,IL,60647.0,...,Pass,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,41.910399,-87.690207,"{'longitude': '41.910398978211525', 'latitude'...",NaN,NaN,NaN,NaN,NaN
2,2316054,SUBWAY,SUBWAY,2590201.0,Restaurant,Risk 1 (High),2512 W NORTH AVE,CHICAGO,IL,60647.0,...,Pass,40. PERSONAL CLEANLINESS - Comments: Inspecto...,41.910399,-87.690207,"{'longitude': '41.910398978211525', 'latitude'...",NaN,NaN,NaN,NaN,NaN
3,2316057,LITTLE GENIUS COMMUNITY DAYCARE 11,LITTLE GENIUS COMMUNITY DAYCARE,2359451.0,Daycare (2 - 6 Years),Risk 1 (High),1000 W 103RD ST,CHICAGO,IL,60643.0,...,Pass,"55. PHYSICAL FACILITIES INSTALLED, MAINTAINED ...",41.706982,-87.647758,"{'longitude': '41.706982259265786', 'latitude'...",NaN,NaN,NaN,NaN,NaN
